In [1]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

from googletrans import Translator
translator = Translator()

import pandas as pd
import time

In [2]:
def translate_tr(text):
    translator = Translator()

    try:
        tr = translator.translate(text, src='tr').text
        return tr
    except:
        return None
translate_tr ('Merhaba')

In [3]:
translator = Translator()

translator.translate("mother", src="en", dest = "ru").text

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def translate(text,from_lang, to_lang):
    tr = None
    attempts = 0
    while not tr:
        try:
            tr = translator.translate(text, src=from_lang, dest = to_lang).text
            return tr
        except:
            tr = None
            time.sleep(0.01)
            attempts +=1
            print("attempt", attempts)
            if attempts > 15:
                tr = "failes"
translate ('I like wine', 'en','fr')

In [4]:
word_list = []
sence_id_list = []
definition = []
examples = []
def_trans = []
ex_trans = []
def sence_translation( from_lang, to_lang, word,words, sence_ids, defs, ex, definitions_trans, examples_trans, debug = False, show_sentence = False,show_all = False):
    sence_id = 0 
    for i, synset in enumerate(wn.synsets(word)):
        words.append(word)
        sence_ids.append(sence_id)
        if (debug): print(i, synset)
        comparison_data = []    
        current_definition = synset.definition()
        defs.append(current_definition)   
        #def_tr = translate (current_definition, from_lang,to_lang)
        def_tr = None
        definitions_trans.append(def_tr)
        examples =  synset.examples()
        comparison_data = [ex_snetence for ex_snetence in examples]

        #comparison_data.append(current_definition)#теперь сюда входят и примеры и само значение
        
        examples = []
        translated_examples = []
        for comp_sentence in comparison_data:
            examples.append(comp_sentence)
            #ex_tr = translate (comp_sentence, from_lang,to_lang)
            ex_tr = None
            translated_examples.append(ex_tr)
            if (debug):print(comp_sentence)
        ex.append(examples)
        examples_trans.append(translated_examples)
        if (debug):print()
        
        sence_id += 1
            
sence_translation("en", "ru", "home",word_list,sence_id_list,definition,examples,def_trans, ex_trans, debug = False, show_sentence = True)#True False
df = pd.DataFrame(list(zip(word_list,sence_id_list,definition,examples,def_trans,ex_trans)), 
                                              columns =['word','sence_id','definition','example','def_trans','ex_trans']) 
df.head(10)

,word,sence_id,definition,example,def_trans,ex_trans
0,home,0,where you live at a particular time,"[deliver the package to my home, he doesn't ha...",None,"[None, None, None]"
1,home,1,housing that someone is living in,"[he built a modest dwelling near the pond, the...",None,"[None, None]"
2,home,2,the country or state or city where you live,[Canadian tariffs enabled United States lumber...,None,"[None, None]"
3,home,3,(baseball) base consisting of a rubber slab wh...,[he ruled that the runner failed to touch home],None,[None]
4,home,4,the place where you are stationed and from whi...,[],None,[]
5,home,5,place where something began and flourished,[the United States is the home of basketball],None,[None]
6,home,6,an environment offering affection and security,"[home is where the heart is, he grew up in a g...",None,"[None, None, None]"
7,home,7,a social unit living together,"[he moved his family to Virginia, It was a goo...",None,"[None, None, None, None]"
8,home,8,an institution where people are cared for,[a home for the elderly],None,[None]
9,home,9,"provide with, or send to, a home",[],None,[]


In [ ]:
df.to_csv("break_wordnet.csv")

In [5]:
words_for_eval = ['go','report','drive','look','feel','people','learn','table','bokk']
def get_multisence_trans(words_list,export_name):
    word_list = []
    sence_id_list = []
    definition = []
    examples = []
    def_trans = []
    ex_trans = []
    for word in words_list:
        sence_translation("en", "ru",word,word_list,sence_id_list,definition,examples,def_trans, ex_trans, debug = False, show_sentence = True)#True False
    df = pd.DataFrame(list(zip(word_list,sence_id_list,definition,examples,def_trans,ex_trans)), 
                                              columns =['word','sence_id','definition','example','def_trans','ex_trans']) 
    df.to_csv(export_name + '.csv')

get_multisence_trans(words_for_eval,'en_fr_wordnet')    
    

In [ ]:
def lesk( word, sentence, debug = False, show_sentence = False,show_all = False):
    
    
    """Ваш код тут"""
    sentence =  sentence.split(" ")
    sentence = [w for w in sentence if not w in stop_words] 
    wordnet_lemmatizer = WordNetLemmatizer()
    sentence = [wordnet_lemmatizer.lemmatize(w) for w in sentence]
    
    bestsense = 0
    #maxoverlap = 0
    min_mean_overlap_dist = 100#начальная точка новой метрики
    final_definition = ''
    
    word_index = sentence.index(word)#индекс исследуемого слова в предложении
    
    #получаем часть речи для дальнейшего отграничения ранжира поиска
    #pos_sentence = nltk.pos_tag(sentence)
    #word_pos = get_pos(pos_sentence, word)
    #if (debug): print("word_pos", word_pos)
    
    for i, synset in enumerate(wn.synsets(word)):
        #if (debug): print(i, synset)
        comparison_data = []    
        current_definition = synset.definition()
        
        definition = tokenize(current_definition)
        #print(definition)
        #if (debug):print("DEFINITION FULL",current_definition)
        
        #if (debug):print("DEFINITION PROCESSED",definition)
        
        
        examples =  synset.examples()
        comparison_data = [tokenize(ex_snetence) for ex_snetence in examples]
        sentence_set = set(sentence)
        #if (show_sentence): print("SETENCE",  sentence)
        
        #if (debug): print(definition & sentence_set)
        mean_overlap_distance = 0
        comparison_data.append(definition)#теперь сюда входят и примеры и само значение
        #print(comparison_data) 
        comparison_data_set = [set(el) for el in comparison_data]
        if (show_sentence): print("SETENCE",  sentence)
        if (debug): print("START ITERATING OVER AL DATA")
        
        word_distance = []
        for comp_sentence_set,comp_sentence in zip(comparison_data_set, comparison_data):
            if (debug):print("comp_sentence:",' '.join(comp_sentence))
            overlap_words_set = sentence_set & comp_sentence_set
            if(word in overlap_words_set):
                overlap_words_set.remove(word)

            if (overlap_words_set and debug):print("overlap_words_set",overlap_words_set)
            for overlapped_word in overlap_words_set:
                if (debug): print("disambig_word_index = ", word_index)
                ind = sentence.index(overlapped_word)
                if (debug):print("overlapped_word = ", overlapped_word)
                if (debug): print("overlapped_word_ind", ind)
                abs_distance = abs(ind - word_index)
                if (debug): print("abs_distance", abs_distance)
                word_distance.append(abs_distance)  
        
        
        #overlap_len = len(overlap)
        if (debug): print("word_distance" , word_distance)
        #применяем новую метрику
        mean_overlap_distance = np.mean(word_distance) / len(word_distance)
        if (debug):
            if ( mean_overlap_distance == 0 or math.isnan(mean_overlap_distance) ): 
                print("ITERATING FINISHED.NO OVERLAP FOUND",'\n')
        if(mean_overlap_distance > 0):
            if (debug):print("OVERLAP_FOUND")
            if (debug):print("EXAMPLES_ORIGINAL = ", examples)
            if (debug):print("ALL_COMPAR_DATA = ", comparison_data)
            if (debug): print(i, synset)
            if (debug):print("DEFINITION FULL",current_definition)
            if (debug):print("DEFINITION PROCESSED",definition)
            if (show_sentence): print("SETENCE",  sentence)
            if (debug): print("mean_overlap_distance", mean_overlap_distance)
                
            if (debug): print("\n")
        
        #обновляем посчитанное значение если оно меньше предыдущего
        if (mean_overlap_distance < min_mean_overlap_dist):
            min_mean_overlap_dist = mean_overlap_distance
            bestsense = i
            final_definition = current_definition
    
    return bestsense, final_definition
lesk("break", "They break the rules in order to convince the rule-makers",debug = True, show_sentence = True)#True False